# Radiotherapy to OMOP

What should I do with `Radiotherapy performed location` in the `radiotherapy` table since I don't know how this information can be retrieved?

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
radiotherapy_IDEA4RC = pd.read_csv("./IDEA4RC-data/radiotherapyIDEA4RC.csv")
radiotherapy_IDEA4RC.head(5)

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

Radiotherapy performed location???
What to do with it?

### Radiotherapy to Episode

In [ ]:
df_tables=radiotherapy_IDEA4RC
df_tables['idThisEvent'] = None
sql = """
INSERT INTO EPISODE (episode_id,person_id, episode_start_date, episode_end_date, episode_concept_id,episode_type_concept_id, parent_episode_id)
VALUES (%s,%s,%s,%s,%s,%s,%s)
"""

queryPerson="""
    SELECT e.patient_id FROM cancer_episode e
    INNER JOIN episode_event v ON e.id=v.cancer_episode
    INNER JOIN systemic_treatment st ON st.episode_event=v.id
    WHERE st.id=%s
"""    

queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""
for idx, row in df_tables.iterrows():
    curIDEA.execute(queryPerson, (row['id']))
    res=curIDEA.fetchone()
    person_id=res[0]
    curIDEA.execute(queryDate, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    dateAux=res
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    parent_episode_id=cur.fetchone()
    episode_start_date=res
    newId=datetime.now().strftime('%Y%m%d%H%M%S')+ str(uuid4())
    row['idThisEvent']=newId
    cur.execute(sql,(newId,person_id, row['startdate'], row['enddate'],row['treatmentResponse'],row['treatmentResponse'],parent_episode_id))
    conn.commit()


### Radiotherapy to Procedure Ocurrence

In [ ]:
queryEVPO="""
    INSERT INTO omopcdm.episode_event (episode_id,event_id,episode_event_field_concept_id)
    VALUES (%s, %s, %s)
"""

sqlProcedure="""
    INSERT INTO omopcdm.procedure_ocurrence (procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_type_concept_id,provider_id)
    VALUES (%s, %s, %s,%s,%s)
"""
def toTableEpisodeEventProcedureOcurrence(idEpisode , idProcedureOcurrence):
    cur.execute(queryEVPO,(idEpisode,idProcedureOcurrence,1147082)) #1147082 or 1147810, not clear
    conn.commit()

for idx, row in df_tables.iterrows():
    curIDEA.execute(queryPerson, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    procedure_concept_id=row['Surgery type']
    procedure_date=row['Date of surgery']
    provider_id=row['Radiotherapy performed location']
    procedure_type_concept_id=0 #Still dont know what to do with this one
    newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
    toTableEpisodeEventProcedureOcurrence(row['idThisEvent'],newId)
    cur.execute(sqlProcedure,(newId, person_id, procedure_concept_id,procedure_date,procedure_type_concept_id))
    conn.commit()

### Radiotherapy to Measurement

Measurement concept id???

meas_event_field_concept_id???

In [ ]:
sqlMeasuementNumber = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_number,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s,%s)
    """
measurement_type_concept_id=38000280
measurement_concept_id=0
columns=[
    "primaryOnly",
    "neckOnly",
    "primaryAndIpsilateralNeck",
    "primaryAndBilateralNeck",
    "distantMetastasis",
    "lung",
    "mediastinum",
    "bone",
    "softTissue",
    "liver",
    "treatmentCompletedAsPlanned",
    "beamQuality",
    "otherSpecify",
    "treatmentTechnique",
    "totalDoseGy",
    "numberOfFractions",
    "adaptiveRt",
    "igrt"
]

booleans=[
    "adaptiveRt",
    "igrt",
    "primaryOnly",
    "neckOnly",
    "primaryAndIpsilateralNeck",
    "primaryAndBilateralNeck",
    "distantMetastasis",
    "lung",
    "mediastinum",
    "bone",
    "softTissue",
    "liver"
]
boolValues=[4188539,4188540]
sqlCode="""
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_concept_id,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
    """
for idx, row in df_tables.iterrows():
    for column in columns:
        curIDEA.execute(queryPerson, (row['Episode Event Reference']))
        res=curIDEA.fetchone()
        person_id=res
        measurement_concept_id = 4037631 if column == 'numberOfFractions' else 0  #Not sure about this one, what to do if not?
        measurement_value=row[column]
        date=row['startDate']
        measurement_event_id=row['idThisEvent']
        if column=='totalDoseGy' or column=='numberOfFractions':
            value_as_number=measurement_value
            cur.execute(sqlMeasuementNumber,(person_id, measurement_concept_id, date, measurement_type_concept_id,value_as_number,measurement_event_id,"SOME CODE")) #WE NEED THE CODE
            conn.commit()
        else:
            if(column in booleans):
                if(measurement_value not in boolValues):
                    measurement_value = 4188540 if measurement_value == 0 else 4188539
                
            cur.execute(sqlCode, (person_id, measurement_concept_id, date, measurement_type_concept_id,measurement_value,measurement_event_id, "SOME CODE")) #WE NEED THE CODE
            conn.commit()

### Radiotherapy to Observation

In [ ]:
sql="""
    INSERT INTO omopcdm.observation (person_id, observation_concept_id, observation_date, observation_type_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
    """
observation_type_concept_id=37117814
for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    observation_concept_id=row['intent']
    observation_date=row['startDate']    
    cur.execute(sql,(person_id, observation_concept_id, observation_date, observation_type_concept_id))
    conn.commit()